Чтобы запустить следующие блокноты, если вы еще этого не сделали, вам нужно развернуть модель, которая использует `text-embedding-ada-002` в качестве базовой модели, и установить имя развертывания в файле .env как `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Далее мы загрузим Индекс Встраивания в DataFrame Pandas. Индекс Встраивания хранится в JSON-файле под названием `embedding_index_3m.json`. Индекс Встраивания содержит встраивания для каждой из расшифровок YouTube до конца октября 2023 года.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Далее мы создадим функцию под названием `get_videos`, которая будет искать в Индексе Встраиваний по запросу. Функция будет возвращать 5 лучших видео, наиболее похожих на запрос. Функция работает следующим образом:

1. Сначала создаётся копия Индекса Встраиваний.
2. Затем вычисляется Встраивание для запроса с помощью OpenAI Embedding API.
3. После этого в Индексе Встраиваний создаётся новый столбец под названием `similarity`. Столбец `similarity` содержит косинусное сходство между Встраиванием запроса и Встраиванием каждого сегмента видео.
4. Далее Индекс Встраиваний фильтруется по столбцу `similarity`. В Индекс Встраиваний включаются только видео с косинусным сходством, большим или равным 0.75.
5. Наконец, Индекс Встраиваний сортируется по столбцу `similarity`, и возвращаются 5 лучших видео.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Эта функция очень простая, она просто выводит результаты поискового запроса.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Сначала индекс встраивания загружается в Dataframe Pandas.  
2. Затем пользователю предлагается ввести запрос.  
3. После этого вызывается функция `get_videos` для поиска по индексу встраивания по запросу.  
4. Наконец, вызывается функция `display_results` для отображения результатов пользователю.  
5. Затем пользователю снова предлагается ввести запрос. Этот процесс продолжается, пока пользователь не введет `exit`.  

![](../../../../translated_images/ru/notebook-search.1e320b9c7fcbb0bc.png)  

Вам будет предложено ввести запрос. Введите запрос и нажмите Enter. Приложение вернет список видео, которые соответствуют запросу. Приложение также вернет ссылку на место в видео, где находится ответ на вопрос.  

Вот несколько запросов для проверки:  

- Что такое Azure Machine Learning?  
- Как работают сверточные нейронные сети?  
- Что такое нейронная сеть?  
- Могу ли я использовать Jupyter Notebooks с Azure Machine Learning?  
- Что такое ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия по обеспечению точности, имейте в виду, что автоматический перевод может содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется использовать профессиональный перевод, выполненный человеком. Мы не несем ответственности за любые недоразумения или неправильные толкования, возникшие в результате использования данного перевода.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
